In [1]:
%run Global.ipynb
%run Helper_Functions.ipynb

In [2]:
train_df_ohe_selected = pd.read_csv(DATASET_DIR + "/train_df_ohe_selected_50.csv", keep_default_na=False)

In [3]:
y = train_df_ohe_selected.iloc[:,-1]
X = train_df_ohe_selected.iloc[:,:-1]

# Kernel SVM

In [4]:
from sklearn import svm

In [5]:
RBF_svm = svm.SVC(kernel='rbf')

In [ ]:
# tune hyperparameter C and gamma
param_grid = {'C': range(1, 101, 25), 'gamma': [0.1, 0.3, 0.5, 0.7, 0.9]}
tune_hyperparameter(RBF_svm, param_grid, X, y, cv=3)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END .....................................C=1, gamma=0.1; total time= 1.8min
[CV] END .....................................C=1, gamma=0.1; total time= 1.8min
[CV] END .....................................C=1, gamma=0.1; total time= 1.8min
[CV] END .....................................C=1, gamma=0.3; total time= 2.3min
[CV] END .....................................C=1, gamma=0.3; total time= 2.8min
[CV] END .....................................C=1, gamma=0.3; total time= 2.8min
[CV] END .....................................C=1, gamma=0.5; total time= 2.6min
[CV] END .....................................C=1, gamma=0.5; total time= 2.3min
[CV] END .....................................C=1, gamma=0.5; total time= 2.4min
[CV] END .....................................C=1, gamma=0.7; total time= 2.6min
[CV] END .....................................C=1, gamma=0.7; total time= 2.5min
[CV] END .....................................C=

In [ ]:
RBF_svm = svm.SVC(kernel='rbf', C=10, gamma=0.1)
RBF_svm.fit(X, y)
cross_val_report(RBF_svm, X, y)[1]
# Kernel SVM performs slightly better on weighted avg precision & recall than Linear SVM

In [ ]:
# Kernel SVM is way too slow with the full feature set - let's do dimensionality reduction
from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=5, random_state=30027)
X_reduced = tsvd.fit_transform(X)
X_reduced

In [ ]:
RBF_svm = svm.SVC(kernel='rbf', C=10, gamma=0.1)
RBF_svm.fit(X_reduced, y)
cross_val_report(RBF_svm, X_reduced, y)[1]

In [ ]:
selector = SelectFromModel(RBF_svm, prefit=True).fit(X, y)
print("RBF SVM Coefficients:", selector.estimator_.coef_)
print("Selector threshold:", selector.threshold_)
selected_features = [X.columns[i] for i in range(X.shape[1]) if selector.get_support()[i]]
print(len(selected_features), "features selected")
X_filtered = X[selected_features]

In [ ]:
RBF_svm = svm.SVC(kernel='rbf', C=10, gamma=0.1)
RBF_svm.fit(X_filtered, y)
cross_val_report(RBF_svm, X_filtered, y)[1]